## Data Visualization

We will be using the same dataset acquired from two different endpoints.
- OpenML: More friendly experience to use with sklearn
- Tensorflow Datasets: Easier to use within TensorFlow for deep learning approaches.

Despite the multiple sources, our data is the exact same across both sources, and these visualizations
represent both sources.

### Load MNIST Dataset for Visualization

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

(train_ds, test_ds), ds_info = tfds.load('mnist', split=['train', 'test'], with_info=True, as_supervised=True, shuffle_files=True)

example = train_ds.take(1)

# Showcasing single handwritten digit image in numpy's NdArray and its
# truth label from the training dataset.
for image, label in tfds.as_numpy(example):  # example is (image, label)
  print(image.shape, label)

### Visualize 9 digits from the training datasets

- Plotting all 784 pixels into 9 (28x28) square images representing the first 9 digits

In [ ]:
fig = tfds.show_examples(train_ds, ds_info)

# Initial Naive Classification Approaches

The following approaches are naive because each of these models were first built with arbitrarily selected parameters based on previous experience with building models on other datasets. If the initial performance is well, then we pursue hyperparameter training

- Logistic Regression Classifier
- Naive Bayes Classifier
- Decision Trees Classifier
- K-Nearest Neighbors Classifier
- Support Vector Machines Classifier

### Load Dataset for SKLearn & Initial Naive Models

In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')

In [ ]:
print(len(mnist.data))
mnist.data.head()

### Visualize Subset of Data

- Plotting all 784 pixels in a 28 x 28 subplot for the first 6 images from the entire dataset.

In [ ]:
import matplotlib.pyplot as plt

image = mnist.data.to_numpy()
plt.subplot(431)
plt.imshow((image[0].reshape(28,28)), cmap=plt.cm.gray_r, interpolation='nearest')
plt.subplot(432)
plt.imshow(image[1].reshape(28,28), cmap=plt.cm.gray_r, interpolation='nearest')
plt.subplot(433)
plt.imshow(image[3].reshape(28,28), cmap=plt.cm.gray_r, interpolation='nearest')
plt.subplot(434)
plt.imshow(image[4].reshape(28,28), cmap=plt.cm.gray_r, interpolation='nearest')
plt.subplot(435)
plt.imshow(image[5].reshape(28,28), cmap=plt.cm.gray_r, interpolation='nearest')
plt.subplot(436)
plt.imshow(image[6].reshape(28,28), cmap=plt.cm.gray_r, interpolation='nearest')

## Split Up Data into Training & Test

We are splitting 

In [ ]:
from sklearn.model_selection import train_test_split

# Step 2: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    mnist.data, mnist.target, test_size=10_000, random_state=42)

# Step 3: Preprocess the data (divide every pixel value by 255)
X_train_scaled = X_train / 255.0
X_test_scaled = X_test / 255.0

## Initial Model Development

- Models are built with arbitrary parameters based on prior experience with building models.

### Logistic Regression Classifier

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Step 4: Create a logistic regression model
# Random State is given specifically for reproducability
lr = LogisticRegression(multi_class='multinomial', random_state=42)

# Step 5: Perform hyperparameter tuning using GridSearchCV
lr_pg = {
    'C': [0.001, 0.01, 0.1, 1, 10],
    'solver': ['newton-cg', 'lbfgs', 'sag', 'saga']  # Different solvers
}  # Hyperparameter grid
lr_gs = GridSearchCV(lr, lr_pg, cv=5, verbose=1, n_jobs=-1)
lr_gs.fit(X_train_scaled, y_train)

# Get the best hyperparameters
lr_bp = lr_gs.best_params_

# Step 6: Evaluate the model on the testing set
lr_bm = lr_gs.best_estimator_
lr_acc = lr_gs.score(X_test_scaled, y_test)

In [ ]:
import pandas as pd
lr_gs_res = pd.DataFrame(lr_gs.cv_results_)
lr_gs_res = lr_gs_res.sort_values(by='rank_test_score')

lr_gs_res.head(1)


#### Logistic Regression Best Model

- Showcase best parameters
- Showcase accuracy, precision, recall

In [ ]:
from sklearn.metrics import classification_report

print(lr_gs.best_params_)
# Same as cross-entropy loss which is the recommended function to use
# for multi-nomial class label classification problems
print("Accuracy via log_loss", lr_acc)
lr_y_pred = lr_bm.predict(X_test_scaled)

# Generate classification report
lr_class_rep = classification_report(y_test, lr_y_pred)
print(lr_class_rep)

### Gaussian Naive Bayes Classifier

There are no hyperparamters / parameters to tune.

Naive Bayes gives pretty low accuracy because it consideres all the probablities as independent of each other (hence-naive).

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

# Step 2: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    mnist.data, mnist.target, test_size=10_000, random_state=42)

# Step 3: Preprocess the data (divide every pixel value by 255)
X_train_scaled = X_train / 255.0
X_test_scaled = X_test / 255.0

# Step 4: Create a Naive Bayes model
nb = GaussianNB()

# Step 5: Perform hyperparameter tuning (No hyperparameters to tune for Naive Bayes)
# You can skip this step for Naive Bayes

# Step 6: Fit the model
nb.fit(X_train_scaled, y_train)

# Step 7: Evaluate the model on the testing set
nb_acc = nb.score(X_test_scaled, y_test)

In [ ]:
print("Accuracy via log_loss", nb_acc)
nb_y_pred = nb.predict(X_test_scaled)

# Generate classification report
nb_class_rep = classification_report(y_test, nb_y_pred)
print(nb_class_rep)

### Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Step 3: Create a Decision Trees model
dt = DecisionTreeClassifier(random_state=42)

# Step 4: Perform hyperparameter tuning using GridSearchCV
dt_pg = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}  # Hyperparameter grid

dt_gs = GridSearchCV(dt, dt_pg, cv=5, verbose=1, n_jobs=-1)
dt_gs.fit(X_train, y_train)

# Get the best hyperparameters
dt_bp = dt_gs.best_params_

# Step 5: Evaluate the model on the testing set
dt_bm = dt_gs.best_estimator_
dt_acc = dt_gs.score(X_test, y_test)

In [ ]:
import pandas as pd
dt_gs_res = pd.DataFrame(dt_gs.cv_results_)
dt_gs_res = dt_gs_res.sort_values(by='rank_test_score')

dt_gs_res.head(1)

#### Decision Tree Best Model

- Showcase best parameters
- Showcase accuracy, precision, recall

In [ ]:
print("Decision Tree Best Parameters", dt_bp)
print("Accuracy via log_loss", dt_acc)
dt_y_pred = dt_bm.predict(X_test_scaled)

# Generate classification report
dt_class_rep = classification_report(y_test, dt_y_pred)
print(dt_class_rep)

### KNN Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Step 4: Create a KNN model
knn = KNeighborsClassifier()

# Step 5: Perform hyperparameter tuning using GridSearchCV
knn_pg = {
    'n_neighbors': range(98, 100, 2),
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'p': [1, 2]  # 1 for Manhattan distance, 2 for Euclidean distance
}  # Hyperparameter grid

knn_gs = GridSearchCV(knn, knn_pg, cv=5, verbose=10, n_jobs=-1)
knn_gs.fit(X_train, y_train)

# Get the best hyperparameters
knn_bp = knn_gs.best_params_

# Step 6: Evaluate the model on the testing set
knn_bm = knn_gs.best_estimator_
knn_acc = knn_gs.score(X_test, y_test)


In [ ]:
import pandas as pd
knn_gs_res = pd.DataFrame(knn_gs.cv_results_)
knn_gs_res = knn_gs_res.sort_values(by='rank_test_score')

knn_gs_res.head(1)

#### KNN Best Model

- Showcase best parameters
- Showcase accuracy, precision, recall

In [ ]:
print("KNN Best Parameters", knn_bp)
print("Accuracy via log_loss", knn_acc)
knn_y_pred = knn_bm.predict(X_test_scaled)

# Generate classification report
knn_class_rep = classification_report(y_test, knn_y_pred)
print(knn_class_rep)

### Support Vector Machine (SVM) Classification Model

In [ ]:
from sklearn.svm import SVC

base_svm = SVC(C=1.0, kernel='rbf')
base_svm.fit(X_train, y_train)
acc = base_svm.score(X_test,y_test)
print(acc)

In [ ]:
from sklearn.svm import SVC

# Step 4: Create a SVM model
svm = SVC()

# Step 5: Perform hyperparameter tuning using GridSearchCV
svm_pg = {
    'C': [0.1, 1, 10, 100],  # Regularization parameter
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': ['scale', 'auto']  # Kernel coefficient
}  # Hyperparameter grid

svm_gs = GridSearchCV(svm, svm_pg, cv=5, verbose=1, n_jobs=-1)
svm_gs.fit(X_train, y_train)

# Get the best hyperparameters
svm_bp = svm_gs.best_params_

# Step 6: Evaluate the model on the testing set
svm_bm = svm_gs.best_estimator_
svm_acc = svm_gs.score(X_test, y_test)

In [ ]:
import pandas as pd
svm_gs_res = pd.DataFrame(svm_gs.cv_results_)
svm_gs_res = svm_gs_res.sort_values(by='rank_test_score')

svm_gs_res.head(1)

#### SVM Best Model

- Showcase best parameters
- Showcase accuracy, precision, recall

In [ ]:
print("SVM Best Parameters", svm_bp)
print("Accuracy via log_loss", svm_acc)
svm_y_pred = svm_bp.predict(X_test_scaled)

# Generate classification report
svm_class_rep = classification_report(y_test, svm_y_pred)
print(svm_class_rep)